In [1]:
import pandas as pd
import os

In [2]:
df_paths = ['Data/pickles/clean_data/{}'.format(file) for file in os.listdir('Data/pickles/clean_data')]

df = pd.concat([
    pd.read_pickle(df_paths[0]),
    pd.read_pickle(df_paths[1])],
axis=1)

In [3]:
df.drop(['echoBuffer',
         'cardLast4Digits',
         'accountOpenDate',
         'transactionDateTime',
         'currentExpDate',
         'customerId',
         'dateOfLastAddressChange',
         'accountNumber',
         'enteredCVV',
         'cardCVV',
         ], inplace=True, axis=1)

# Data I am working with for predictive modeling
df.isnull().sum()

expirationDateKeyInMatch       0
isFraud                        0
merchantCategoryCode           0
merchantCountryCode          724
merchantName                   0
posConditionCode             409
posEntryMode                4054
transactionAmount              0
transactionType              698
acqCountry                  4562
availableMoney                 0
cardPresent                    0
creditLimit                    0
currentBalance                 0
dtype: int64

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler

In [5]:
ros = RandomUnderSampler()
y = df['isFraud']
df.drop('isFraud', inplace=True, axis=1)
new_x, new_y = ros.fit_resample(df, y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y)

pipeline = ColumnTransformer([
    ('cat_pipe', Pipeline([
        ('cat_imputer', SimpleImputer(strategy='most_frequent')),
        ('one_hot', OneHotEncoder(handle_unknown='ignore'))
    ]),
     ['merchantCountyCode',
      'posConditionCode',
      'posEntryMode',
      'transactionType',
      'acqCountry',
      '']),
], remainder='passthrough')
# for col in ['merchantCountyCode', 'posConditionCode', 'posEntryMode']